![image-2.png](attachment:image-2.png)
_Aprendizaje Automático_

_Máster Universitario en Inteligencia Artificial_

# Actividad grupal: Explorando patrones ocultos: Detección de Anomalías en datos
## Objetivos

Mediante este trabajo se pretende que ponga en práctica la aplicación de las técnicas de detección de anomalías. El objetivo es analizar si existen anomalías en el conjunto de datos aplicando técnicas univariantes y multivariantes. Debe detallar los pasos que hay que realizar para detección de valores inusuales.

- 	Entender los métodos media móvil, z-score, Isolation Forest y Local Outlier Factor
-	Implementar el código para cada una de estas técnicas y analizar si existen o no anomalía
-   Investigar sobre una técnica de detección de anomalías con un caso de uso concreto.

Descripción de la actividad

Debes realizar la actividad en el Notebook adjunto. Debes ir completando el código solicitado, así como responder a las preguntas que se van realizando.

Tareas que debe realizar
- Detección de anomalías.
- Debes completar el código pedido en este notebook y contestar las preguntas finales.
- Buscar dos artículos científicos (https://scholar.google.es/) con un caso de uso de detección de anomalías. Los artículos deben estar en revistas científicas, y deben ser posteriores a 2015. No debe utilizar técnicas de Deep- Learning.Para cada artículo debe indicar:
    * Objetivo: cuál es el objetivo de la investigación, es decir a qué problema real está aplicando la detección anomalías.
    * Técnicas de detección de anomas empleadas.
    * Principales resultados de la aplicación y de la investigación.


#### Integrantes del grupo

In [2]:
# Coloca en este espacio el nombre de los integrantes del grupo
# Luis Alberto López Gonźalez

## Detección de anomalías

Los transformadores juegan un papel muy importante en el sistema eléctrico. Aunque son algunos de los componentes más confiables de la red eléctrica, también son propensos a fallar debido a muchos factores, tanto internos como externos. Podría haber muchos iniciadores que causen una falla en el transformador, pero los que potencialmente pueden conducir a una falla catastrófica son los siguientes:
Falla mecánica
Falla dieléctrica

Contenido
Estos datos se recopilaron a través de dispositivos IoT desde el 25 de junio de 2019 hasta el 14 de abril de 2020 y se actualizaron cada 15 minutos.
El dataset contiene 19352 registros.
Descripción de los parámetros:
Voltaje de corriente:
1. VL1- Fase Línea 1
2. VL2- Fase Línea 2
3. VL3- Fase Línea 3
4. IL1- Línea actual 1
5. IL2- Línea actual 2
6. IL3- Línea actual 3
7. VL12- Línea de tensión 1 2
8. VL23- Línea de tensión 2 3
9. VL31- Línea de tensión 3 1
10. INUT-Corriente neutra

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# se carga el dataset, con separados ;, se transforman las dos columnas Date y Time a una única columna con tipo datetime
# en el fichero CSV existen NaN identificados por ?
df = pd.read_csv('../data/raw/CurrentVoltage.csv', sep=',') 

# Verificar la carga de datos
print(df.head())


    DeviceTimeStamp    VL1    VL2    VL3  IL1  IL2  IL3  VL12  VL23  VL31  \
0  2019-06-25T13:06    0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   
1  2019-06-27T10:49  238.7  238.7  238.8  0.0  0.0  0.0   0.0   0.0   0.0   
2  2019-06-27T10:51  238.4  238.5  238.5  0.0  0.0  0.0   0.0   0.0   0.0   
3  2019-06-27T10:52  239.9  240.0  240.0  0.0  0.0  0.0   0.0   0.0   0.0   
4  2019-06-27T10:52  239.9  240.0  240.0  0.0  0.0  0.0   0.0   0.0   0.0   

   INUT  
0   0.0  
1   0.0  
2   0.0  
3   0.0  
4   0.0  


#### ¿Cuántas instancias tiene el dataset?

In [ ]:
# Pon aquí tu código
print("Número de instancias y número de atributos")
print(df.shape)

Número de instancias y número de atributos
(19352, 11)


#### ¿Cuál es el tipo de datos predominante en las variables?

In [ ]:
#Inserta acá el código
print("Tipo de datos predominantes en las columnas")
print(df.dtypes)

Tipo de datos de las columnas
DeviceTimeStamp     object
VL1                float64
VL2                float64
VL3                float64
IL1                float64
IL2                float64
IL3                float64
VL12               float64
VL23               float64
VL31               float64
INUT               float64
dtype: object


El tipo predominante en las variables es 'float64'


## Variables nulas

#### ¿Tiene este dataset datos faltantes?

¿Tiene este dataset datos faltantes?

In [5]:
# Identificación de datos faltantes
print("Datos faltantes por columna")
print(df.isnull().sum())


Datos faltantes por columna
DeviceTimeStamp    0
VL1                0
VL2                0
VL3                0
IL1                0
IL2                0
IL3                0
VL12               0
VL23               0
VL31               0
INUT               0
dtype: int64


#### ¿Es necesario hacer algún tratamiendo adicional sobre los datos?

In [55]:
### PON AQUÍ TU CÓDIGO EN CASO DE QUE SEA NECESARIO HACER ALGÚN TRATAMIENTO ADICIONAL

In [6]:
# Convertir DeviceTimeStamp a datetime
df["DeviceTimeStamp"] = pd.to_datetime(df["DeviceTimeStamp"])

# Confirmar tipos de datos
print(df.dtypes)



DeviceTimeStamp    datetime64[ns]
VL1                       float64
VL2                       float64
VL3                       float64
IL1                       float64
IL2                       float64
IL3                       float64
VL12                      float64
VL23                      float64
VL31                      float64
INUT                      float64
dtype: object


In [7]:
# Verificar y eliminar duplicados
print(f"Duplicados antes: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)
print(f"Duplicados después: {df.duplicated().sum()}")



Duplicados antes: 32
Duplicados después: 0


In [8]:
import numpy as np

# Definir umbrales de corte (percentiles)
for col in df.select_dtypes(include=["float64"]).columns:
    low, high = df[col].quantile([0.01, 0.99])
    df[col] = np.clip(df[col], low, high)



In [9]:
from sklearn.preprocessing import StandardScaler

# Escalar columnas numéricas
scaler = StandardScaler()
num_cols = df.select_dtypes(include=["float64"]).columns
df[num_cols] = scaler.fit_transform(df[num_cols])



In [10]:
# Creación de variables adicionales
# Crear variable de diferencia temporal
df["time_diff"] = df["DeviceTimeStamp"].diff().dt.total_seconds()

# Crear variables agregadas por hora
df["hour"] = df["DeviceTimeStamp"].dt.hour



In [12]:
print(df.head())
df.describe()


      DeviceTimeStamp       VL1       VL2       VL3       IL1       IL2  \
0 2019-06-25 13:06:00 -2.458741 -2.454606 -2.573555 -1.807223 -1.472833   
1 2019-06-27 10:49:00 -0.289935 -0.232612 -0.154324 -1.807223 -1.472833   
2 2019-06-27 10:51:00 -0.324915 -0.256504 -0.192523 -1.807223 -1.472833   
3 2019-06-27 10:52:00 -0.150012 -0.077311 -0.001531 -1.807223 -1.472833   
5 2019-06-27 10:53:00 -0.010089  0.066044  0.151263 -1.807223 -1.472833   

        IL3      VL12      VL23      VL31      INUT  time_diff  hour  
0 -1.927038 -3.192624 -3.199311 -3.193824 -1.773258        NaN    13  
1 -1.927038 -3.192624 -3.199311 -3.193824 -1.773258   164580.0    10  
2 -1.927038 -3.192624 -3.199311 -3.193824 -1.773258      120.0    10  
3 -1.927038 -3.192624 -3.199311 -3.193824 -1.773258       60.0    10  
5 -1.927038 -3.192624 -3.199311 -3.193824 -1.773258       60.0    10  


,DeviceTimeStamp,VL1,VL2,VL3,IL1,IL2,IL3,VL12,VL23,VL31,INUT,time_diff,hour
count,19320,1.932000e+04,1.932000e+04,1.932000e+04,1.932000e+04,1.932000e+04,1.932000e+04,1.932000e+04,1.932000e+04,1.932000e+04,1.932000e+04,1.931900e+04,19320.000000
mean,2019-11-06 15:41:14.388198656,9.415059e-16,2.212539e-15,-3.718948e-15,-1.588791e-16,3.530647e-17,-1.471103e-16,3.530647e-16,-1.176882e-16,-4.707530e-17,-5.884412e-17,1.312503e+03,11.810559
min,2019-06-25 13:06:00,-2.458741e+00,-2.454606e+00,-2.573555e+00,-1.807223e+00,-1.472833e+00,-1.927038e+00,-3.192624e+00,-3.199311e+00,-3.193824e+00,-1.773258e+00,0.000000e+00,0.000000
25%,2019-08-22 06:56:15,-6.747230e-01,-6.626751e-01,-6.254377e-01,-6.569421e-01,-6.831060e-01,-6.213490e-01,1.739656e-01,1.878559e-01,1.797203e-01,-6.329412e-01,9.000000e+02,6.000000
50%,2019-10-27 17:52:30,1.764754e-01,1.735595e-01,1.385297e-01,-1.001963e-01,-2.182665e-01,-6.805161e-02,3.167600e-01,3.135878e-01,3.139907e-01,-4.591439e-02,9.000000e+02,12.000000
75%,2020-01-29 18:18:45,7.361674e-01,7.230850e-01,6.733070e-01,6.478541e-01,5.914538e-01,7.126996e-01,3.982369e-01,3.895332e-01,3.945530e-01,6.423240e-01,9.000000e+02,17.000000
max,2020-04-14 00:30:00,1.762270e+00,1.774351e+00,1.895655e+00,2.486390e+00,2.712746e+00,2.281726e+00,5.267519e-01,5.184801e-01,5.237882e-01,2.597798e+00,2.903940e+06,23.000000
std,NaN,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,1.000026e+00,2.454964e+04,6.703193


#### Media móvil

In [70]:
# Vamos a aplicar la técnica de detección de anomalías de Media Móvil sobre la variable INUT

In [3]:
# Calcula la media móvil

In [4]:
# Define un umbral para detectar anomalías

In [ ]:
# Visualiza los resultados

#### Z-Score

In [68]:
# Vamos a aplicar la técnica de detección de anomalías Z-Score
# Pon aquí tu código

In [ ]:
# Define un umbral para detectar anomalías (por ejemplo, 3 desviaciones estándar)

In [ ]:
# Visualiza los resultados

#### Isolation Forest

In [57]:
# Vamos a aplicar la técnica de detección de anomalías de Isolation Forest, configura el parámetro contamination en 5%
# Pon aquí tu código

In [71]:
# Agrega las columnas de puntaje y si es dato anómalo o no en el dataframe

In [72]:
# Crea un gráfico de barras para mostrar los datos anómalos vs los datos normales

In [73]:
# Crea un gráfico donde se puedan ver los datos anómalos sobre la variable INUT.

In [74]:
# Repite los pasos anteriores con el parámetro de contamination en 1%

## Local Outlier Factor

In [78]:
# Vamos a aplicar la técnica de detección de anomalías de Local Outlier Factor
# Pon aquí tu código

In [79]:
# Crea un gráfico de barras para mostrar los datos anómalos vs los datos normales

In [80]:
# Crea un gráfico donde se puedan ver los datos anómalos sobre la variable INUT.

## Análisis

In [81]:
# Realiza un análisis detallado de los resultados obtenidos en cada una de las técnicas aplicadas. Utiliza gráficos o trablas para apoyar tu conclusiones

## Artículos de investigación

#### Artículo 1

In [82]:
# Coloca en este espacio el análisis del artículo 1

#### Artículo 2

In [83]:
# Coloca en este espacio el análisis del artículo 2